<a href="https://colab.research.google.com/github/TUIlmenauAMS/InteractiveSpectrogramWithZoom/blob/main/spectrogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Interactive Spectrogram with Zoom

Load the audio signal for the spectrogram using the "Files" panel on the left, then copy its path into the text field argument below, and run that cell. Verify by listening to its mono version by clicking on the playback button below.

Then in the "Spectrogram" cell you can change the FFT lenght of the Short Time Fourier Transfrom (STFT) for the spectrogram, and the number of times the STFT windows overlap, if desired. Then run the cell. The spectrogram has "hover", zoom, and "save" tools. 

In [83]:
import scipy.io.wavfile as wav 
import numpy as np
from IPython.display import Audio, display

filename = '/content/Procession - chan 1 clip.wav' #@param {'type': 'string'}
rate, snd = wav.read(filename)
print("Data Type:", snd.dtype, "rate=", rate)

#Convert to multichannel
if len(snd.shape)==1 : #mono
  #mono=np.around((snd[:,0]/2+snd[:,1]/2)).astype(np.int16)
  snd=np.expand_dims(snd, axis=1) #add channels dimensio

channels=snd.shape[1]
print("channels=", channels)
siglen=len(snd)/rate

print("Listen to it:")
for chan in range(channels):
  print("channel "+str(chan)+":")
  display(Audio(snd[:,chan],rate=rate))


Data Type: int16 rate= 44100
channels= 1
Listen to it:
channel 0:


In [84]:
#@title Spectrogram with Tools {run: "auto" }
import scipy.signal as signal
import matplotlib.pyplot as plt

import bokeh.io
bokeh.io.output_notebook()
import bokeh.plotting
from bokeh.palettes import Spectral11
#from bokeh.models import HoverTool
from bokeh.plotting import figure

#Example: https://docs.bokeh.org/en/2.4.0/docs/gallery/image.html

#hovertool: https://docs.bokeh.org/en/latest/docs/user_guide/tools.html#hovertool

fftlen= "8192" #@param [256,2048, 8192]
fftlen=int(fftlen)
timesoverlap= "2" #@param [2, 4, 8, 16 ] 
#This many timer are the windows overlapped
timesoverlap=int(timesoverlap)

noverlap=fftlen -fftlen//timesoverlap

for chan in range(channels):
  freq,t,y = signal.stft(snd[:,chan],fs=rate, nperseg=fftlen, noverlap = noverlap) 
  magdb=20*np.log10(np.abs(y)) #Values in dB

  print("max(magdb)=",np.max(magdb), "min(magdb)=", np.min(magdb), "magdb.shape=",magdb.shape )

  p = figure(title='Spectrogram channel '+str(chan), x_axis_label= 'Time (s)', y_axis_label='Frequency (Hz)',tooltips=[("time", "$x"), ("freq.:", "$y"), ("dB", "@image")])
  #p.x_range.range_padding = p.y_range.range_padding = 0
  p.image(image=[magdb], x=0, y=0, dw=siglen, dh=rate/2, palette="Spectral11", level="image")
  p.grid.grid_line_width = 0.5
  bokeh.plotting.show(p)



max(magdb)= 70.54554 min(magdb)= -85.29184 magdb.shape= (4097, 65)
